In [ ]:
import time
import warnings
import sys
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectPercentile, f_classif, SelectFromModel
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve, auc, mean_squared_error, \
    r2_score, mean_absolute_error,cohen_kappa_score,accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, XGBClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import multiprocessing

start = time.time()
warnings.filterwarnings("ignore")

def standardize(col):
    return (col - np.mean(col)) / np.std(col)

# the metrics for classification
def statistical(y_true, y_pred, y_pro):
    c_mat = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = list(c_mat.flatten())
    se = tp / (tp + fn)
    sp = tn / (tn + fp)
    auc_prc = auc(precision_recall_curve(y_true, y_pro, pos_label=1)[1],
                  precision_recall_curve(y_true, y_pro, pos_label=1)[0])
    acc = (tp + tn) / (tn + fp + fn + tp)
#     acc_skl = accuracy_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pro)
    recall = se
#     recall_skl = recall_score(y_true, y_pred)
    precision = tp / (tp + fp)
#     precision_skl = precision_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall) # F1 = 2 * (precision * recall) / (precision + recall)
#     f1_skl = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + 1e-8)
#     mcc_skl = matthews_corrcoef(y_true,y_pred)
    return tn,fp,fn,tp,se,sp,auc_prc,acc,auc_roc,recall,precision,f1,kappa,mcc

def all_one_zeros(data):
    if (len(np.unique(data)) == 2):
        flag = False
    else:
        flag = True
    return flag

feature_selection = False

In [ ]:
modelName= 'svm'
tasks = 'logBCF'
data_file_1 = './data/BCF1859/BCF1859-SEED'
data_file_2 = '-ECFP-group.csv'
file_name = data_file_1+'0'+data_file_2
dataset_label = 'BCF1859_ECFP'
hyper_file_name = './model/' + dataset_label + '_'+modelName+'_hyperopt_info.csv'
muti_tasks = ['logBCF']
ecfp = True
data_preprocessing = True
task_type = 'reg'  # 'reg' or 'cla'
OPT_ITERS = 50
repetitions = 10
patience = 50
num_pools = 10
GPUNum = 0
space_ = {'C': hp.uniform('C', 0, 50), 'gamma': hp.uniform('gamma', 0, 0.2)}
dataset = pd.read_csv(file_name)
pd_res = []

In [ ]:
def hyper_runing(subtask):
    
    sub_dataset = dataset.drop(columns='SMILES')

    if data_preprocessing :
        # detect the na in the subtask (y cloumn)
        rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
        sub_dataset.drop(index=rm_index, inplace=True)

        # remove the features with na
        sub_dataset = sub_dataset.dropna(axis=1)
        # *******************
        # demension reduction
        # *******************
        # Removing features with low variance
        # threshold = 0.05
        data_fea_var = sub_dataset.iloc[:, 2:].var()
        del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
        sub_dataset.drop(columns=del_fea1, inplace=True)

        # pair correlations
        # threshold = 0.95
        data_fea_corr = sub_dataset.iloc[:, 2:].corr()
        del_fea2_col = []
        del_fea2_ind = []
        length = data_fea_corr.shape[1]
        for i in range(length):
            for j in range(i + 1, length):
                if abs(data_fea_corr.iloc[i, j]) >= 0.95:
                    del_fea2_col.append(data_fea_corr.columns[i])
                    del_fea2_ind.append(data_fea_corr.index[j])
        sub_dataset.drop(columns=del_fea2_ind, inplace=True)

    # standardize the features
    cols_ = list(sub_dataset.columns)[2:]
    if not ecfp :
        sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # get the attentivefp data splits
    data_tr = sub_dataset[sub_dataset['group'] == 'train']
    data_va = sub_dataset[sub_dataset['group'] == 'valid']
    data_te = sub_dataset[sub_dataset['group'] == 'test']

    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 2:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 2:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 2:].values)

    if feature_selection:
        # univariate feature selection
        trans1 = SelectPercentile(f_classif, percentile=80)
        trans1.fit(data_tr_x, data_tr_y)
        data_tr_x = trans1.transform(data_tr_x)
        data_va_x = trans1.transform(data_va_x)
        data_te_x = trans1.transform(data_te_x)

        # select from model
        clf = XGBClassifier(n_jobs=12, random_state=1)
        clf = clf.fit(data_tr_x, data_tr_y)
        trans2 = SelectFromModel(clf, prefit=True)

        data_tr_x = trans2.transform(data_tr_x)
        data_va_x = trans2.transform(data_va_x)
        data_te_x = trans2.transform(data_te_x)

    num_fea = data_tr_x.shape[1]
    print('the num of retained features for the ' + dataset_label + ' ' + subtask + ' is:', num_fea)

    def hyper_opt(args):
        model = svm.SVC(**args, kernel='rbf', random_state=1, probability=True,
                        cache_size=2000, max_iter=10000) if task_type == 'cla' else \
            svm.SVR(**args, kernel='rbf', cache_size=2000, max_iter=10000)
        model.fit(data_tr_x, data_tr_y)
        val_preds = model.predict_proba(data_va_x) if task_type == 'cla' else model.predict(data_va_x)
        loss = 1 - roc_auc_score(data_va_y, val_preds[:, 1]) if task_type == 'cla' else np.sqrt(
            mean_squared_error(data_va_y, val_preds))
        return {'loss': loss, 'status': STATUS_OK}

    # start hyper-parameters optimization
    trials = Trials()
    best_results = fmin(hyper_opt, space_, algo=tpe.suggest, max_evals=OPT_ITERS, trials=trials, show_progressbar=False)
    print('the best hyper-parameters for ' + dataset_label + ' ' + subtask + ' are:  ', best_results)
    best_model = svm.SVC(C=best_results['C'], gamma=best_results['gamma'], kernel='rbf', random_state=1,
                         probability=True,  cache_size=2000, max_iter=10000) \
        if task_type == 'cla' else svm.SVR(C=best_results['C'], gamma=best_results['gamma'], kernel='rbf',
                                           cache_size=2000, max_iter=10000)
    best_model.fit(data_tr_x, data_tr_y)
    num_of_compounds = len(sub_dataset)

    if task_type == 'cla':
        # training set
        tr_pred = best_model.predict_proba(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds, data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0],
                      best_results['C'],
                      best_results['gamma']]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))

        # validation set
        va_pred = best_model.predict_proba(data_va_x)
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds, data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0],
                      best_results['C'],
                      best_results['gamma']]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = best_model.predict_proba(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds, data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0],
                      best_results['C'],
                      best_results['gamma']]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = best_model.predict(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      best_results['C'],
                      best_results['gamma'],
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = best_model.predict(data_va_x)
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      best_results['C'],
                      best_results['gamma'],
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = best_model.predict(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      best_results['C'],
                      best_results['gamma'],
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results


pool = multiprocessing.Pool(num_pools)
res = pool.starmap(hyper_runing, zip(muti_tasks))
pool.close()
pool.join()
for item in res:
    for i in range(3):
        pd_res.append(item[i])
if task_type == 'cla':
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'num_of_retained_feature',
                                               'num_of_compounds', 'postives',
                                               'negtives', 'negtives/postives',
                                               'C', 'gamma',
                                               'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                               'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'num_of_retained_feature',
                                               'num_of_compounds', 'C', 'gamma', 'rmse', 'r2', 'mae'])
best_hyper.to_csv( hyper_file_name, index=0)

In [ ]:
if task_type == 'cla':
    print('train', best_hyper[best_hyper['set'] == 'tr']['auc_roc'].mean(), best_hyper[best_hyper['set'] == 'tr']['auc_prc'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['auc_roc'].mean(), best_hyper[best_hyper['set'] == 'va']['auc_prc'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['auc_roc'].mean(), best_hyper[best_hyper['set'] == 'te']['auc_prc'].mean())
else:
    print('train', best_hyper[best_hyper['set'] == 'tr']['rmse'].mean(), best_hyper[best_hyper['set'] == 'tr']['r2'].mean(), best_hyper[best_hyper['set'] == 'tr']['mae'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['rmse'].mean(), best_hyper[best_hyper['set'] == 'va']['r2'].mean(), best_hyper[best_hyper['set'] == 'va']['mae'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['rmse'].mean(), best_hyper[best_hyper['set'] == 'te']['r2'].mean(), best_hyper[best_hyper['set'] == 'te']['mae'].mean())

In [ ]:
def sp_model_runing(data_tr,data_va,data_te,best_hyper,split):
    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 2:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 2:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 2:].values)

    if feature_selection:
        # univariate feature selection
        trans1 = SelectPercentile(f_classif, percentile=80)
        trans1.fit(data_tr_x, data_tr_y)
        data_tr_x = trans1.transform(data_tr_x)
        data_va_x = trans1.transform(data_va_x)
        data_te_x = trans1.transform(data_te_x)

        # select from model
        clf = XGBClassifier(n_jobs=6, random_state=1)
        clf = clf.fit(data_tr_x, data_tr_y)
        trans2 = SelectFromModel(clf, prefit=True)

        data_tr_x = trans2.transform(data_tr_x)
        data_va_x = trans2.transform(data_va_x)
        data_te_x = trans2.transform(data_te_x)

    num_fea = data_tr_x.shape[1]
    model = svm.SVC(C=best_hyper[best_hyper.subtask == subtask].iloc[0,]['C'],
                    gamma=best_hyper[best_hyper.subtask == subtask].iloc[0,]['gamma'], kernel='rbf', random_state=1,
                    probability=True,
                    cache_size=2000, max_iter=10000) \
        if task_type == 'cla' else svm.SVR(
        C=best_hyper[best_hyper.subtask == subtask].iloc[0,]['C'],
        gamma=best_hyper[best_hyper.subtask == subtask].iloc[0,]['gamma'],
        kernel='rbf', cache_size=2000, max_iter=10000)

    model.fit(data_tr_x, data_tr_y)
    num_of_compounds = sub_dataset.shape[0]
    
    import pickle
    pickle.dump(model, open("./model/"+modelName+"_"+str(split)+"_"+dataset_label+".pkl", "wb"))
    
    if task_type == 'cla':
        # training set
        tr_pred = model.predict_proba(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0]]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))

        # validation set
        va_pred = model.predict_proba(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0]]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = model.predict_proba(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0]]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = model.predict(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = model.predict(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = model.predict(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results    

In [ ]:
pd_res = []

for split in range(1, 11):
    # 10 repetitions based on thr best hypers
    file_name = data_file_1+str(split)+data_file_2
    best_hyper = pd.read_csv(hyper_file_name)
    sub_dataset = pd.read_csv(file_name)
    subtask = tasks

    if data_preprocessing:
        # detect the NA in the subtask (y cloumn)
        rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
        sub_dataset.drop(index=rm_index, inplace=True)

        # remove the features with na
        sub_dataset = sub_dataset.dropna(axis=1)

        # *******************
        # demension reduction
        # *******************
        # Removing features with low variance
        # threshold = 0.05
        data_fea_var = sub_dataset.iloc[:, 3:].var()
        del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
        sub_dataset.drop(columns=del_fea1, inplace=True)

        # pair correlations
        # threshold = 0.95
        data_fea_corr = sub_dataset.iloc[:, 3:].corr()
        del_fea2_col = []
        del_fea2_ind = []
        length = data_fea_corr.shape[1]
        for i in range(length):
            for j in range(i + 1, length):
                if abs(data_fea_corr.iloc[i, j]) >= 0.95:
                    del_fea2_col.append(data_fea_corr.columns[i])
                    del_fea2_ind.append(data_fea_corr.index[j])
        sub_dataset.drop(columns=del_fea2_ind, inplace=True)
    # standardize the features
    cols_ = list(sub_dataset.columns)[3:]
    if not ecfp :
        sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # get the data splits
    data_tr = sub_dataset[sub_dataset['group'] == 'train'].drop(columns='SMILES')
    data_va = sub_dataset[sub_dataset['group'] == 'valid'].drop(columns='SMILES')
    data_te = sub_dataset[sub_dataset['group'] == 'test'].drop(columns='SMILES')
    
    res = sp_model_runing(data_tr,data_va,data_te,best_hyper,split)
    for i in range(3):
        pd_res.append(res[i])

In [ ]:
#data_fea_var

In [ ]:
#data_fea_corr

In [ ]:
#cols_

In [ ]:
if task_type == 'cla':
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'postives',
                                             'negtives', 'negtives/postives',
                                             'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                             'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'rmse', 'r2', 'mae'])
stat_res.to_csv('./model/' + dataset_label + '_'+modelName+'_statistical_results_split.csv', index=0)
stat_res

In [ ]:
args = {'data_label': dataset_label, 'metric': 'auc_roc' if task_type == 'cla' else 'rmse', 'model': 'RF'}
print('{}_{}: the mean {} for the training set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 'r2', np.mean(
        stat_res[stat_res['set'] == 'tr']['r2']), np.std(
        stat_res[stat_res['set'] == 'tr']['r2'])))
print(
    '{}_{}: the mean {} for the validation set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 'r2', np.mean(
            stat_res[stat_res['set'] == 'va']['r2']), np.std(
            stat_res[stat_res['set'] == 'va']['r2'])))
print('{}_{}: the mean {} for the test set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                             'r2', np.mean(
        stat_res[stat_res['set'] == 'te']['r2']), np.std(
        stat_res[stat_res['set'] == 'te']['r2'])))

end = time.time()  # get the end time

In [ ]:
# acc auc_roc recall precision f1 kappa mcc
rmse_str = 'rmse of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['rmse']), 
                np.std(stat_res[stat_res['set'] == 'tr']['rmse']),
                np.mean(stat_res[stat_res['set'] == 'va']['rmse']), 
                np.std(stat_res[stat_res['set'] == 'va']['rmse']),
                np.mean(stat_res[stat_res['set'] == 'te']['rmse']), 
                np.std(stat_res[stat_res['set'] == 'te']['rmse']),
)
r2_str = 'r2 of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['r2']), 
                np.std(stat_res[stat_res['set'] == 'tr']['r2']),
                np.mean(stat_res[stat_res['set'] == 'va']['r2']), 
                np.std(stat_res[stat_res['set'] == 'va']['r2']),
                np.mean(stat_res[stat_res['set'] == 'te']['r2']), 
                np.std(stat_res[stat_res['set'] == 'te']['r2']),
)
mae_str = 'mae of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['mae']), 
                np.std(stat_res[stat_res['set'] == 'tr']['mae']),
                np.mean(stat_res[stat_res['set'] == 'va']['mae']), 
                np.std(stat_res[stat_res['set'] == 'va']['mae']),
                np.mean(stat_res[stat_res['set'] == 'te']['mae']), 
                np.std(stat_res[stat_res['set'] == 'te']['mae']),
)
print('the elapsed time is:', (end - start)/3600, 'H')
print(rmse_str)
print(r2_str)
print(mae_str)

In [ ]:
import pandas as pd
import collections
dict1 = {
         "Model: "+modelName+" ":['rmse ','r2','mae'],
         "Train":[
                  format(np.mean(stat_res[stat_res['set'] == 'tr']['rmse']), '.3f'),
                  format(np.mean(stat_res[stat_res['set'] == 'tr']['r2']), '.3f'),
                  format(np.mean(stat_res[stat_res['set'] == 'tr']['mae']), '.3f')
                 ],
         "Tr_STD":[
                   format(np.std(stat_res[stat_res['set'] == 'tr']['rmse']), '.3f'),
                   format(np.std(stat_res[stat_res['set'] == 'tr']['r2']), '.3f'),
                   format(np.std(stat_res[stat_res['set'] == 'tr']['mae']), '.3f')
                  ],
         "Validation":[
                       format(np.mean(stat_res[stat_res['set'] == 'va']['rmse']), '.3f'),
                       format(np.mean(stat_res[stat_res['set'] == 'va']['r2']), '.3f'),
                       format(np.mean(stat_res[stat_res['set'] == 'va']['mae']), '.3f')
                    ],
         "Va_STD":[
                   format(np.std(stat_res[stat_res['set'] == 'va']['rmse']), '.3f'),
                   format(np.std(stat_res[stat_res['set'] == 'va']['r2']), '.3f'),
                   format(np.std(stat_res[stat_res['set'] == 'va']['mae']), '.3f')
                  ],
         "Test":[
                 format(np.mean(stat_res[stat_res['set'] == 'te']['rmse']), '.3f'),
                 format(np.mean(stat_res[stat_res['set'] == 'te']['r2']), '.3f'),
                 format(np.mean(stat_res[stat_res['set'] == 'te']['mae']), '.3f')
                ],
          "Te_STD":[
                    format(np.std(stat_res[stat_res['set'] == 'te']['rmse']), '.3f'),
                    format(np.std(stat_res[stat_res['set'] == 'te']['r2']), '.3f'),
                    format(np.std(stat_res[stat_res['set'] == 'te']['mae']), '.3f')
                   ]
        }
dict1 = collections.OrderedDict(dict1)
df = pd.DataFrame(dict1,index = None)
df.to_csv('output/'+dataset_label+'_output_'+modelName+'.csv',index = False)
df